In [1]:
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
from config import db_password
from flask import Flask
from flask_sqlalchemy import sqlalchemy
import os
import re
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [2]:
path = 'Resources/'
degrees = pd.read_csv(path +'degrees_clean.csv', index_col=0)
degrees

,object_id,degree_type,subject,institution,graduated_at
0,p:6117,MBA,NaN,NaN,NaN
1,p:6136,BA,"English, French","Washington University, St. Louis",1/1/1990
2,p:6136,MS,Mass Communication,Boston University,1/1/1992
3,p:6005,MS,Internet Technology,University of Greenwich,1/1/2006
4,p:5832,BCS,"Computer Science, Psychology",Rice University,NaN
...,...,...,...,...,...
109605,p:268589,CPA,NaN,American Institute of Certtified Public Accoun...,NaN
109606,p:268527,MS & BS,Engineering,South China University of Technology,NaN
109607,p:268527,PhD,Engineering,Clarkson University,NaN
109608,p:268528,B.S.,Electrical Engineering,Colorado State University,NaN


In [11]:
degrees_red = degrees[['object_id', 'degree_type']]
degrees_red

,object_id,degree_type
0,p:6117,MBA
1,p:6136,BA
2,p:6136,MS
3,p:6005,MS
4,p:5832,BCS
...,...,...
109605,p:268589,CPA
109606,p:268527,MS & BS
109607,p:268527,PhD
109608,p:268528,B.S.


In [5]:
objects = pd.read_csv(path +'objects_clean.csv', index_col=0)
objects

C:\Users\phsstudent\anaconda3\envs\mlenv\lib\site-packages\IPython\core\interactiveshell.py:3072: DtypeWarning: Columns (3,5,7,8,9,10,11,13,14,17,18) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,object_id,entity_type,parent_id,name,category_code,status,founded_at,closed_at,country_code,state_code,city,region,first_investment_at,last_investment_at,investment_rounds,invested_companies,first_funding_at,last_funding_at,funding_rounds,funding_total_usd,first_milestone_at,last_milestone_at,milestones,relationships
0,c:1,Company,NaN,Wetpaint,web,operating,2005-10-17,NaN,USA,WA,Seattle,Seattle,NaN,NaN,0,0,2005-10-01,2008-05-19,3,39750000.0,2010-09-05,2013-09-18,5,17
1,c:10,Company,NaN,Flektor,games_video,acquired,NaN,NaN,USA,CA,Culver City,Los Angeles,NaN,NaN,0,0,NaN,NaN,0,0.0,NaN,NaN,0,6
2,c:100,Company,NaN,There,games_video,acquired,NaN,NaN,USA,CA,San Mateo,SF Bay,NaN,NaN,0,0,NaN,NaN,0,0.0,2003-02-01,2011-09-23,4,12
3,c:10000,Company,NaN,MYWEBBO,network_hosting,operating,2008-07-26,NaN,NaN,NaN,NaN,unknown,NaN,NaN,0,0,NaN,NaN,0,0.0,NaN,NaN,0,0
4,c:10001,Company,NaN,THE Movie Streamer,games_video,operating,2008-07-26,NaN,NaN,NaN,NaN,unknown,NaN,NaN,0,0,NaN,NaN,0,0.0,NaN,NaN,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
462646,r:9995,Product,c:14164,"SiteLink, listing feed for Brokerages",NaN,operating,NaN,NaN,NaN,NaN,NaN,unknown,NaN,NaN,0,0,NaN,NaN,0,0.0,NaN,NaN,0,0
462647,r:9996,Product,c:14164,"EDCLink, listing feed for Economic Development...",NaN,operating,NaN,NaN,NaN,NaN,NaN,unknown,NaN,NaN,0,0,NaN,NaN,0,0.0,NaN,NaN,0,0
462648,r:9997,Product,c:14164,"Cmail, broadcast email marketing",NaN,operating,NaN,NaN,NaN,NaN,NaN,unknown,NaN,NaN,0,0,NaN,NaN,0,0.0,NaN,NaN,0,0
462649,r:9998,Product,c:14164,"CatylistCRM, contact database",NaN,operating,NaN,NaN,NaN,NaN,NaN,unknown,NaN,NaN,0,0,NaN,NaN,0,0.0,NaN,NaN,0,0


In [6]:
obj_people = objects[objects['object_id'].str.contains('p:', regex=True)]

In [15]:
obj_people_red = obj_people[['object_id', 'entity_type', 'name']]

In [25]:
combined = obj_people_red.merge(degrees_red, how='outer')

In [30]:
combined[combined['name'].isnull()]

,object_id,entity_type,name,degree_type
267693,p:6611,NaN,NaN,BS
267694,p:6614,NaN,NaN,BS
267695,p:6614,NaN,NaN,BS
267696,p:6659,NaN,NaN,BS
267697,p:6729,NaN,NaN,BA (Hons) First Class
...,...,...,...,...
268241,p:20063,NaN,NaN,NaN
268242,p:2739,NaN,NaN,Msc
268243,p:2739,NaN,NaN,Bsc
268244,p:2739,NaN,NaN,PG


In [43]:
degree_clean = degrees[degrees['object_id'].isin(objects['object_id'])]

In [44]:
degree_clean.to_csv(path + 'degrees_clean.csv')

In [45]:
people = pd.read_csv(path +'people_clean.csv', index_col=0)
people

,object_id,first_name,last_name,affiliation_name
0,p:2,Ben,Elowitz,Blue Nile
1,p:3,Kevin,Flaherty,Wetpaint
2,p:4,Raju,Vegesna,Zoho
3,p:5,Ian,Wenig,Zoho
4,p:6,Kevin,Rose,i/o Ventures
...,...,...,...,...
226703,p:268587,Mark,Stolper,Unaffiliated
226704,p:268589,John,Pins,Unaffiliated
226705,p:268590,David,Schulhof,Unaffiliated
226706,p:268592,Matthew,D. Rosen,Unaffiliated


In [3]:
people = pd.read_csv(path +'people_clean.csv', index_col=0)
people

,object_id,first_name,last_name,affiliation_name
0,p:2,Ben,Elowitz,Blue Nile
1,p:3,Kevin,Flaherty,Wetpaint
2,p:4,Raju,Vegesna,Zoho
3,p:5,Ian,Wenig,Zoho
4,p:6,Kevin,Rose,i/o Ventures
...,...,...,...,...
226703,p:268587,Mark,Stolper,Unaffiliated
226704,p:268589,John,Pins,Unaffiliated
226705,p:268590,David,Schulhof,Unaffiliated
226706,p:268592,Matthew,D. Rosen,Unaffiliated


In [6]:
people_clean = people[people['object_id'].isin(objects['object_id'])]
people_clean

,object_id,first_name,last_name,affiliation_name
0,p:2,Ben,Elowitz,Blue Nile
1,p:3,Kevin,Flaherty,Wetpaint
2,p:4,Raju,Vegesna,Zoho
3,p:5,Ian,Wenig,Zoho
4,p:6,Kevin,Rose,i/o Ventures
...,...,...,...,...
226703,p:268587,Mark,Stolper,Unaffiliated
226704,p:268589,John,Pins,Unaffiliated
226705,p:268590,David,Schulhof,Unaffiliated
226706,p:268592,Matthew,D. Rosen,Unaffiliated


In [8]:
people_clean.to_csv(path + 'people_cleaner.csv')
print('Save Complete')

Save Complete


# Relationships

In [52]:
rels = pd.read_csv(path+'relationships.csv')
rels_df = rels.drop(columns={'id', 'relationship_id', 'created_at', 'updated_at'})
rels_df

,person_object_id,relationship_object_id,start_at,end_at,is_past,sequence,title
0,p:2,c:1,NaN,NaN,0,8,Co-Founder/CEO/Board of Directors
1,p:3,c:1,NaN,NaN,1,279242,VP Marketing
2,p:4,c:3,NaN,NaN,0,4,Evangelist
3,p:5,c:3,2006-03-01,2009-12-01,1,4,Senior Director Strategic Alliances
4,p:7,c:4,2005-07-01,2010-04-05,1,1,Chief Executive Officer
...,...,...,...,...,...,...,...
402873,p:268527,c:286218,2005-01-01,2010-01-01,1,1,Sr. Consultant/Patent Agent
402874,p:268597,c:12612,NaN,NaN,0,1,"VP, GM & Editor In Chief - Clinical Decison Su..."
402875,p:268528,c:286152,NaN,NaN,1,273049,Vice President
402876,p:268528,c:286220,NaN,NaN,1,1,"General Counsel, Director of Patents and Licen..."


In [56]:
rels_df['start_at'] = rels_df['start_at'].fillna('1900-01-01')
rels_df

,person_object_id,relationship_object_id,start_at,end_at,is_past,sequence,title
0,p:2,c:1,1900-01-01,NaN,0,8,Co-Founder/CEO/Board of Directors
1,p:3,c:1,1900-01-01,NaN,1,279242,VP Marketing
2,p:4,c:3,1900-01-01,NaN,0,4,Evangelist
3,p:5,c:3,2006-03-01,2009-12-01,1,4,Senior Director Strategic Alliances
4,p:7,c:4,2005-07-01,2010-04-05,1,1,Chief Executive Officer
...,...,...,...,...,...,...,...
402873,p:268527,c:286218,2005-01-01,2010-01-01,1,1,Sr. Consultant/Patent Agent
402874,p:268597,c:12612,1900-01-01,NaN,0,1,"VP, GM & Editor In Chief - Clinical Decison Su..."
402875,p:268528,c:286152,1900-01-01,NaN,1,273049,Vice President
402876,p:268528,c:286220,1900-01-01,NaN,1,1,"General Counsel, Director of Patents and Licen..."


In [57]:
rels_df['end_at']=rels_df['end_at'].fillna('1900-01-01')
rels_df

,person_object_id,relationship_object_id,start_at,end_at,is_past,sequence,title
0,p:2,c:1,1900-01-01,1900-01-01,0,8,Co-Founder/CEO/Board of Directors
1,p:3,c:1,1900-01-01,1900-01-01,1,279242,VP Marketing
2,p:4,c:3,1900-01-01,1900-01-01,0,4,Evangelist
3,p:5,c:3,2006-03-01,2009-12-01,1,4,Senior Director Strategic Alliances
4,p:7,c:4,2005-07-01,2010-04-05,1,1,Chief Executive Officer
...,...,...,...,...,...,...,...
402873,p:268527,c:286218,2005-01-01,2010-01-01,1,1,Sr. Consultant/Patent Agent
402874,p:268597,c:12612,1900-01-01,1900-01-01,0,1,"VP, GM & Editor In Chief - Clinical Decison Su..."
402875,p:268528,c:286152,1900-01-01,1900-01-01,1,273049,Vice President
402876,p:268528,c:286220,1900-01-01,1900-01-01,1,1,"General Counsel, Director of Patents and Licen..."


In [59]:
rels_clean = rels_df[rels_df['person_object_id'].isin(objects['object_id'])]
rels_clean

,person_object_id,relationship_object_id,start_at,end_at,is_past,sequence,title
0,p:2,c:1,1900-01-01,1900-01-01,0,8,Co-Founder/CEO/Board of Directors
1,p:3,c:1,1900-01-01,1900-01-01,1,279242,VP Marketing
2,p:4,c:3,1900-01-01,1900-01-01,0,4,Evangelist
3,p:5,c:3,2006-03-01,2009-12-01,1,4,Senior Director Strategic Alliances
4,p:7,c:4,2005-07-01,2010-04-05,1,1,Chief Executive Officer
...,...,...,...,...,...,...,...
402872,p:268527,c:286217,2010-09-01,2012-04-01,1,1,"Sr. Manager, IP Analytics"
402873,p:268527,c:286218,2005-01-01,2010-01-01,1,1,Sr. Consultant/Patent Agent
402875,p:268528,c:286152,1900-01-01,1900-01-01,1,273049,Vice President
402876,p:268528,c:286220,1900-01-01,1900-01-01,1,1,"General Counsel, Director of Patents and Licen..."


In [60]:
rels_clean2 = rels_clean[rels_clean['relationship_object_id'].isin(objects['object_id'])]
rels_clean2

,person_object_id,relationship_object_id,start_at,end_at,is_past,sequence,title
0,p:2,c:1,1900-01-01,1900-01-01,0,8,Co-Founder/CEO/Board of Directors
1,p:3,c:1,1900-01-01,1900-01-01,1,279242,VP Marketing
2,p:4,c:3,1900-01-01,1900-01-01,0,4,Evangelist
3,p:5,c:3,2006-03-01,2009-12-01,1,4,Senior Director Strategic Alliances
4,p:7,c:4,2005-07-01,2010-04-05,1,1,Chief Executive Officer
...,...,...,...,...,...,...,...
402868,p:268589,c:12612,1900-01-01,1900-01-01,1,2,Accounting Manager for Facts & Comparisons
402869,p:268589,c:12612,1900-01-01,1900-01-01,1,3,"Controller, Wolters Kluwer Health"
402870,p:268589,c:12612,1997-01-01,1900-01-01,1,1,"Director, Accounting - Professional & Educatio..."
402871,p:268527,c:39172,2012-04-01,2013-10-01,1,273047,"Director, Business Development"


In [69]:
rels_clean2[rels_clean2['person_object_id']=='p:80624']

,person_object_id,relationship_object_id,start_at,end_at,is_past,sequence,title
83033,p:80624,c:45066,1961-02-18,1900-01-01,0,2,"Prof. Dr., trustee, presidente di consiglio, U..."
84694,p:80624,c:66405,1961-01-01,1900-01-01,0,1,"Prof. Dr., trustee, presidente di consiglio, U..."


In [72]:
rels_clean2.loc[83033, 'title'].replace(",", "")
rels_clean2.loc[84694, 'title'].replace(",", "")

"Prof. Dr. trustee presidente di consiglio Universita'"

In [73]:
rels_clean2.to_csv(path+'relationships_cleaner.csv')
print('Save Complete')

Save Complete


# Experiment with Sequence Column

In [74]:
rc2 = rels_clean2

In [78]:
rc2[rc2['sequence']==rc2['sequence'].max()]

,person_object_id,relationship_object_id,start_at,end_at,is_past,sequence,title
374286,p:99472,c:29588,1900-01-01,1900-01-01,0,399946,Founder


In [81]:
rc2[rc2['sequence']==rc2['sequence'].max()].groupby(['person_object_id'])

In [98]:
rcA = rc2.groupby(['person_object_id']).sequence.max()
rcA_df = pd.DataFrame(rcA)
rcA_df.reset_index()

,person_object_id,sequence
0,p:10,2
1,p:100,1
2,p:1000,1
3,p:10000,6
4,p:100000,1
...,...,...
200859,p:99992,2
200860,p:99994,1
200861,p:99995,1
200862,p:99997,7


In [99]:
rcB = rc2[['person_object_id','sequence', 'title']]
rcB

,person_object_id,sequence,title
0,p:2,8,Co-Founder/CEO/Board of Directors
1,p:3,279242,VP Marketing
2,p:4,4,Evangelist
3,p:5,4,Senior Director Strategic Alliances
4,p:7,1,Chief Executive Officer
...,...,...,...
402868,p:268589,2,Accounting Manager for Facts & Comparisons
402869,p:268589,3,Controller Wolters Kluwer Health
402870,p:268589,1,Director Accounting - Professional & Education...
402871,p:268527,273047,Director Business Development


In [105]:
rcB.merge(rcA, how='left', on='person_object_id')

,person_object_id,sequence_x,title,sequence_y
0,p:2,8,Co-Founder/CEO/Board of Directors,8
1,p:3,279242,VP Marketing,279248
2,p:4,4,Evangelist,5
3,p:5,4,Senior Director Strategic Alliances,5
4,p:7,1,Chief Executive Officer,8
...,...,...,...,...
400171,p:268589,2,Accounting Manager for Facts & Comparisons,4
400172,p:268589,3,Controller Wolters Kluwer Health,4
400173,p:268589,1,Director Accounting - Professional & Education...,4
400174,p:268527,273047,Director Business Development,273048


In [132]:
rc2_sort = rc2.sort_values(by=['sequence'], ascending=False)
rc2_sort

,person_object_id,relationship_object_id,start_at,end_at,is_past,sequence,title
374286,p:99472,c:29588,1900-01-01,1900-01-01,0,399946,Founder
374287,p:99472,c:37723,1900-01-01,1900-01-01,0,399945,Advisor
135830,p:99472,f:9273,1900-01-01,1900-01-01,0,399944,Managing Partner
296318,p:99472,f:13132,2013-08-06,1900-01-01,0,399943,Director
134057,p:98433,c:158453,2011-10-28,1900-01-01,0,398565,Founder & MD
...,...,...,...,...,...,...,...
16680,p:18104,c:194,1900-01-01,1900-01-01,1,1,Editor of Ostatic.com
16679,p:18103,c:194,1900-01-01,1900-01-01,1,1,Editor of FoundRead
16678,p:18102,c:194,1900-01-01,1900-01-01,1,1,Special Projects Editor
16677,p:11404,c:11018,1900-01-01,1900-01-01,1,1,CEO


In [133]:
rc2_clean = rc2_sort.drop_duplicates(subset=['person_object_id'])
rc2_clean.shape

(200864, 7)

In [131]:
rc2_clean.sort_index()

,person_object_id,relationship_object_id,start_at,end_at,is_past,sequence,title
0,p:2,c:1,1900-01-01,1900-01-01,0,8,Co-Founder/CEO/Board of Directors
17,p:21,c:7299,1900-01-01,1900-01-01,0,1,VP Customer Service
21,p:25,c:7299,1900-01-01,1900-01-01,0,1,VP Product
34,p:40,c:10,1900-01-01,1900-01-01,1,2,Co-founder and CTO
40,p:51,c:14,1900-01-01,1900-01-01,0,1,Director of Strategic Relations
...,...,...,...,...,...,...,...
402853,p:158143,c:12612,1900-01-01,1900-01-01,0,4,President & CEO Wolters Kluwer Health Clinical...
402864,p:268589,c:12612,2006-06-01,1900-01-01,0,4,VP Finance & Clinical Solutions
402865,p:268590,c:286215,1900-01-01,1900-01-01,0,1,Co-Founder
402866,p:268592,c:256200,1900-01-01,1900-01-01,0,1,CEO
